**Clustering the neighborhoods in Toronto**

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

**Scraping Data into DataFrame**

In [2]:
TorontoData =requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
TorontoSoup = BeautifulSoup(TorontoData, 'html.parser')

In [3]:
PostalCode=[]
Borough=[]
Neighborhood=[]


for row in TorontoSoup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if (len(cells)>0):
        PostalCode.append(cells[0].text)
        Borough.append(cells[1].text)
        Neighborhood.append(cells[2].text.rstrip('\n'))

**Creating Data Frame**

In [4]:
TorontoDF = pd.DataFrame({"PostalCode": PostalCode,
                           "Borough": Borough,
                           "Neighborhood": Neighborhood})

#Dropping Cells with Borough Not assigned.
TorontoDataFrame = TorontoDF[TorontoDF.Borough != "Not assigned"].reset_index(drop=True)
TorontoDataFrame.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [5]:
# Grouping Neighborhood in same borough
TorontoDataFrameGrouped = TorontoDataFrame.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
TorontoDataFrameGrouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
# Neighborhood not assigned, make it the same as borough
for index, row in TorontoDataFrameGrouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
TorontoDataFrameGrouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
Column = ["PostalCode", "Borough", "Neighborhood"]
DF = pd.DataFrame(columns=Column)

List = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in List:
    DF = DF.append(TorontoDataFrameGrouped[TorontoDataFrameGrouped["PostalCode"]==postcode], ignore_index=True)
    
DF

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


In [8]:
# Final cleaned up row count and colummns information.
TorontoDataFrameGrouped.shape

(103, 3)

**load the latitude and longitude from the csv file on Coursera**

In [9]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data  
PostalLatLon = pd.read_csv("Geospatial_Coordinates.csv")
PostalLatLon.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
PostalLatLon.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#Joining Postallatlon and TorontoDataFrameGrouped
TorontoDFLatLon = TorontoDataFrameGrouped.merge(PostalLatLon, on="PostalCode", how="left")
TorontoDFLatLon.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
Column= ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
DF = pd.DataFrame(columns=Column)
List = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in List:
    DF = DF.append(TorontoDFLatLon[TorontoDFLatLon["PostalCode"]==postcode], ignore_index=True)
    
DF

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


**Getting the latitude and longitude of Toronto from Geopy**

In [12]:
Place = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(Place)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


**Toronto Map with neighborhoods superimposed on top**

In [13]:
# create map of Toronto using latitude and longitude values
TorontoMap = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(TorontoDFLatLon['Latitude'], TorontoDFLatLon['Longitude'], TorontoDFLatLon['Borough'], TorontoDFLatLon['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(TorontoMap)  
    
TorontoMap

**Filter only boroughs with Toronto Name**

In [14]:
BoroughNames = list(TorontoDFLatLon.Borough.unique())

TorontoBorough = []

for x in BoroughNames:
    if "toronto" in x.lower():
        TorontoBorough.append(x)
        
TorontoBorough

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [15]:
TorontoDFLatLon = TorontoDFLatLon[TorontoDFLatLon['Borough'].isin(TorontoBorough)].reset_index(drop=True)
TorontoDFLatLon.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:
NewTorontoMap = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoDFLatLon['Latitude'], TorontoDFLatLon['Longitude'], TorontoDFLatLon['Borough'], TorontoDFLatLon['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(NewTorontoMap)  
    
NewTorontoMap

**Foursquare API to explore the neighborhoods**

In [17]:
CLIENT_ID = '0PJBJPJO55CEEMDIOUFUOJCKLZSPIVDBHA0GWV115JE2FVDG' 
CLIENT_SECRET = 'GNG43YZ5MOCFEQGOYBBAH05B3Z4WH0XGXGVLDKDLMES05YTH' 
VERSION = '20190619'

In [18]:
radius = 500
LIMIT = 100

Venues = []

for lat, long, post, borough, neighborhood in zip(TorontoDFLatLon['Latitude'], TorontoDFLatLon['Longitude'], TorontoDFLatLon['PostalCode'], TorontoDFLatLon['Borough'], TorontoDFLatLon['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        Venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [19]:
VenueDF = pd.DataFrame(Venues)
VenueDF.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
VenueDF.head()

,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [20]:
VenueDF.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,5,5,5,5,5,5
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"The Beaches West, India Bazaar",18,18,18,18,18,18
M4M,East Toronto,Studio District,38,38,38,38,38,38
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,9,9,9,9,9,9
M4R,Central Toronto,North Toronto West,16,16,16,16,16,16
M4S,Central Toronto,Davisville,38,38,38,38,38,38
M4T,Central Toronto,"Moore Park, Summerhill East",2,2,2,2,2,2


In [21]:
VenueDF['VenueCategory'].unique()[:50]

array(['Trail', 'Health Food Store', 'Pub', 'Other Great Outdoors',
       'Neighborhood', 'Greek Restaurant', 'Ice Cream Shop',
       'Cosmetics Shop', 'Italian Restaurant', 'Brewery', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Pizza Place',
       'Juice Bar', 'Bookstore', 'Restaurant', 'Furniture / Home Store',
       'Spa', 'Grocery Store', 'Diner', 'Bubble Tea Shop',
       'Indian Restaurant', 'Coffee Shop', 'Caribbean Restaurant',
       'Bakery', 'Café', 'American Restaurant', 'Liquor Store',
       'Fish & Chips Shop', 'Burger Joint', 'Gym', 'Park',
       'Sushi Restaurant', 'Burrito Place', 'Pet Store', 'Steakhouse',
       'Movie Theater', 'Fast Food Restaurant', 'Sandwich Place',
       'Fish Market', 'Cheese Shop', 'Comfort Food Restaurant',
       'Chinese Restaurant', 'Middle Eastern Restaurant',
       'Seafood Restaurant', 'Stationery Store', 'Coworking Space', 'Bar',
       'Gastropub'], dtype=object)

**Analyzing each area**

In [22]:
toronto_onehot = pd.get_dummies(VenueDF[['VenueCategory']], prefix="", prefix_sep="")

toronto_onehot['PostalCode'] = VenueDF['PostalCode'] 
toronto_onehot['Borough'] = VenueDF['Borough'] 
toronto_onehot['Neighborhoods'] = VenueDF['Neighborhood'] 

fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1699, 241)


,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0

In [23]:
TG = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = TG['PostalCode']
neighborhoods_venues_sorted['Borough'] = TG['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = TG['Neighborhoods']

for ind in np.arange(TG.shape[0]):
    row_categories = TG.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(38, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Pub,Trail,Neighborhood,Other Great Outdoors,Falafel Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Fruit & Vegetable Store,Juice Bar,Liquor Store,Spa
2,M4L,East Toronto,"The Beaches West, India Bazaar",Park,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
5,M4P,Central Toronto,Davisville North,Playground,Hotel,Clothing Store,Food & Drink Shop,Grocery Store,Park,Gym,Breakfast Spot,Sandwich Place,Falafel Restaurant
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Yoga Studio,Bagel Shop,Park,Clothing Store,Dessert Shop,Chinese Restaurant,Rental Car Location,Diner,Salon / Barbershop
7,M4S,Central Toronto,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Thai Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Deli / Bodega
8,M4T,Central Toronto,"Moore Park, Summerhill East",Tennis Court,Playground,Convenience Store,Doner Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,Liquor Store,Light Rail Station,Sushi Restaurant,Supermarket,Sports Bar,Fried Chicken Joint,American Restaurant,Vietnamese Restaurant


**Cluster Areas**

In [24]:
kclusters = 5

toronto_grouped_clustering = TG.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([0, 0, 0, 0, 3, 0, 0, 0, 2, 0], dtype=int32)

In [25]:
toronto_merged = TorontoDFLatLon.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head()

(38, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Neighborhood,Other Great Outdoors,Falafel Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Fruit & Vegetable Store,Juice Bar,Liquor Store,Spa
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [26]:
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Neighborhood,Other Great Outdoors,Falafel Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant
20,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,Gastropub,Deli / Bodega,Italian Restaurant,Bakery,American Restaurant,Pizza Place
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,American Restaurant,Bakery,Gastropub,Deli / Bodega,Italian Restaurant
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Coffee Shop,Café,Sandwich Place,Pizza Place,Park,Cosmetics Shop,Liquor Store,Jewish Restaurant,Pub,Indian Restaurant
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,0,Café,Italian Restaurant,Japanese Restaurant,Bar,Bookstore,Restaurant,Bakery,College Gym,Chinese Restaurant,Beer Bar
26,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,0,Café,Vegetarian / Vegan Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Bar,Vietnamese Restaurant,Chinese Restaurant,Cocktail Bar
27,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0,Airport Terminal,Airport Lounge,Airport Service,Plane,Sculpture Garden,Boat or Ferry,Boutique,Harbor / Marina,Bar,Airport Gate
19,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Restaurant,Café,Italian Restaurant,Scenic Lookout,Sporting Goods Shop,Brewery,Fried Chicken Joint
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Coffee Shop,Café,Restaurant,Cocktail Bar,Hotel,Italian Restaurant,Beer Bar,Seafood Restaurant,Japanese Restaurant,Bakery
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Baby Store,Nightclub,Diner,Italian Restaurant,Restaurant


**Clusters**

In [27]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Pub,Trail,Neighborhood,Other Great Outdoors,Falafel Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant
20,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Gastropub,Deli / Bodega,Italian Restaurant,Bakery,American Restaurant,Pizza Place
21,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,American Restaurant,Bakery,Gastropub,Deli / Bodega,Italian Restaurant
24,Central Toronto,0,Coffee Shop,Café,Sandwich Place,Pizza Place,Park,Cosmetics Shop,Liquor Store,Jewish Restaurant,Pub,Indian Restaurant
25,Downtown Toronto,0,Café,Italian Restaurant,Japanese Restaurant,Bar,Bookstore,Restaurant,Bakery,College Gym,Chinese Restaurant,Beer Bar
26,Downtown Toronto,0,Café,Vegetarian / Vegan Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Bar,Vietnamese Restaurant,Chinese Restaurant,Cocktail Bar
27,Downtown Toronto,0,Airport Terminal,Airport Lounge,Airport Service,Plane,Sculpture Garden,Boat or Ferry,Boutique,Harbor / Marina,Bar,Airport Gate
19,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Restaurant,Café,Italian Restaurant,Scenic Lookout,Sporting Goods Shop,Brewery,Fried Chicken Joint
28,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Cocktail Bar,Hotel,Italian Restaurant,Beer Bar,Seafood Restaurant,Japanese Restaurant,Bakery
30,Downtown Toronto,0,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Baby Store,Nightclub,Diner,Italian Restaurant,Restaurant


In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Ice Cream Shop,Garden,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Tennis Court,Playground,Convenience Store,Doner Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Jewelry Store,Trail,Park,Sushi Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Yoga Studio
10,Downtown Toronto,4,Park,Playground,Trail,Dog Run,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


**Inference:**

**1. Cluster 1 is filled with business areas, restaurants, supermarkets and cafes.**
**2. Cluster 2 is a greenary zone, Cluster 3 is kind of play area, Cluster 4 is full of swim schools and parks, and Cluster 5 is park trail.**
**3. Among all, Cluster1 is a really big.**